In [9]:
import pandas as pd
from datasets import load_dataset

data = load_dataset("csv", data_files="megablunders.csv", )

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [11]:
def tokenize(examples):
   return tokenizer(examples["original_sentence"], padding="max_length", truncation=True)

data = data.map(tokenize, batched=True)

In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=9)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import numpy as np
import evaluate 

metric = evaluate.load("accuracy")

def compute_metrics(eval_prod):
    loigts, labels = eval_prod
    
    predictions = np.argmax(loigts, axis=-1)
    return metric.compute(predictions=predictions, labels=labels)

In [14]:
from transformers import TrainingArguments

trainingArguments = TrainingArguments(
    output_dir="megablunders",
    eval_strategy="epoch", 
    push_to_hub=False
)

In [ ]:
from transformers import Trainer


from transformers import Trainer
from datasets import DatasetDict

# Split the training data into train and test sets
train_test_split = data["train"].train_test_split(test_size=0.2, seed=42)
data = DatasetDict({
    "train": train_test_split["train"],
    "test": train_test_split["test"]
})

trainer = Trainer(
    model=model,
    args=trainingArguments,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    compute_metrics=compute_metrics
)

trainer.train()

/Users/felixstuart/Documents/School/Clubs/Programming Club/Potential Projects/ML Megablunders/.venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.